In [1]:
import pandas as pd

In [ ]:
def clean_csv(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)
    
    # Standardize column names (lowercase, replace spaces with underscores)
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    # Drop duplicates
    df = df.drop_duplicates()
    
    # Handle missing values (fill with median for numerical, mode for categorical)
    for col in df.columns:
        if df[col].dtype == 'O':  
            df[col] = df[col].fillna(df[col].mode()[0])
        else:
            df[col] = df[col].fillna(df[col].median())
    
    # Convert data types where applicable
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col])
        except ValueError:
            pass  
    
    return df



In [8]:
clean_avg_price=clean_csv("avg_price_data.csv")
clean_avg_price.to_csv("clean_avg_price.csv", index=False)

In [ ]:
#aggregate by year

clean_avg_price["month"] = clean_avg_price["month"].astype(str).str[-4:]
yearly_avg_df = clean_avg_price.groupby("month", as_index=False).mean()



In [26]:
yearly_avg_df.to_csv("yearly_avg_df.csv", index=False)

In [4]:
#clean up downloaded data for minimum wage (https://fred.stlouisfed.org/release/tables?rid=387&eid=243906&od=2005-01-01#)

file_paths =["min_wage1.csv","min_wage2.csv","min_wage3.csv", "min_wage4.csv"]

mega_df = pd.read_csv(file_paths[0]).fillna(float('nan'))

for file_path in file_paths[1:]:
    df = pd.read_csv(file_path).fillna(float('nan'))
    mega_df = mega_df.set_index("observation_date").join(df.set_index("observation_date"), how="outer").reset_index()


mega_df = mega_df.drop_duplicates()


print(mega_df.head())


  observation_date  STTMINWGFG  STTMINWGAK  STTMINWGAR  STTMINWGAZ  \
0       2005-01-01        5.15        7.15        5.15         NaN   
1       2006-01-01        5.15        7.15        5.15         NaN   
2       2007-01-01        5.15        7.15        6.25        6.75   
3       2008-01-01        5.85        7.15        6.25        6.90   
4       2009-01-01        6.55        7.15        6.25        7.25   

   STTMINWGCA  STTMINWGCO  STTMINWGCT  STTMINWGDE  STTMINWGFL  ...  \
0        6.75        5.15        7.10        6.15         NaN  ...   
1        6.75        5.15        7.40        6.15        6.40  ...   
2        7.50        6.85        7.65        6.65        6.67  ...   
3        8.00        7.02        7.65        7.15        6.79  ...   
4        8.00        7.28        8.00        7.15        7.21  ...   

   STTMINWGRI  STTMINWGSD  STTMINWGTX  STTMINWGUT  STTMINWGVA  STTMINWGVT  \
0        6.75        5.15        5.15        5.15        5.15        7.00   
1   

In [5]:
#fix column names
mega_df.columns = [col.replace("STTMINWG", "") if col != "observation_date" else col for col in mega_df.columns]
mega_df = mega_df.rename(columns={"FG": "federal_minimum"})

In [7]:
#add states with no own laws and copy federal min
for state in ["AL", "SC", "TN", "MS", "LA"]:
    mega_df[state] = mega_df["federal_minimum"]

#if no value turn into federal min
for index, row in mega_df.iterrows():
    federal_min = row["federal_minimum"]
    mega_df.loc[index] = row.fillna(federal_min)

In [8]:
#simplify date
mega_df["observation_date"] = mega_df["observation_date"].astype(str).str[:4]


In [9]:
mega_df.to_csv("final_min_wage.csv", index=False)

In [10]:
## create data for heatmap
import json

wage_data = pd.read_csv("final_min_wage.csv")
price_data = pd.read_csv("yearly_avg_price.csv")

# Melt data into long format
wage_data = wage_data.melt(id_vars=["observation_date"], var_name="state", value_name="min_wage")
price_data = price_data.melt(id_vars=["month"], var_name="item", value_name="price")

# Merge data
data = wage_data.merge(price_data, left_on="observation_date", right_on="month")

# Compute purchasing power
data["purchasing_power"] = data["min_wage"] / data["price"]

#fill nan 
data.fillna

# Save to JSON
data = data.to_dict(orient="records")
with open("data.json", "w") as f:
    json.dump(data, f)

